In [1]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3 # Connect S3 bucket
import pandas as pd

sm_boto3 = boto3.client("sagemaker", region_name="us-east-1")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'mobsexampleforsagemaker'
print("Using Bucket:" , bucket)

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\thenu\AppData\Local\sagemaker\sagemaker\config.yaml
Using Bucket: mobsexampleforsagemaker


In [3]:
df = pd.read_csv("mpc_train.csv")

In [4]:
df.shape

(2000, 21)

In [5]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [6]:
df["price_range"].value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [7]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [9]:
df.isnull().sum()*100

battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
dtype: int64

In [10]:
features = list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [11]:
label = features.pop(-1)
label

'price_range'

In [14]:
X = df[features]
Y = df[label]

In [15]:
Y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [17]:
print("X_train_shape", X_train.shape)
print("X_test shape", X_test.shape)
print("Y_train shape", Y_train.shape)
print("Y_test shape:", Y_test.shape)

X_train_shape (1600, 20)
X_test shape (400, 20)
Y_train shape (1600,)
Y_test shape: (400,)


In [18]:
trainX = pd.DataFrame(X_train)
trainX['label'] = Y_train

testX = pd.DataFrame(X_test)
testX['label'] = Y_test

In [19]:
trainX.to_csv('train_v1.csv', index=False)
testX.to_csv('test_v1.csv', index=False)

In [34]:
# send data to S3. SageMaker will take training data from S3
sk_folder = "sagemaker/mobile_price_classification/sklearncontainer"

train_path = sess.upload_data(path="train_v1.csv", bucket=bucket, key_prefix=sk_folder)
test_path = sess.upload_data(path="test_v1.csv", bucket=bucket, key_prefix=sk_folder)


In [21]:
print(train_path)

s3://mobsexampleforsagemaker/sagemaker/mobile_price_classification/sklearncontainer/train_v1.csv


In [22]:
%%writefile script.py

import argparse
import os
import json
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
import joblib
import pathlib
from io import StringIO
import boto3
import pandas as pd
import numpy as np

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__ =='__main__':

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument('--n_estimators', type=int, default=100)
    parser.add_argument('--random_state', type=int, default=0)
    # parser.add_argument('--epochs', type=int, default=10)
    # parser.add_argument('--batch-size', type=int, default=100)
    # parser.add_argument('--learning-rate', type=float, default=0.1)

    # an alternative way to load hyperparameters via SM_HPS environment variable.
    # parser.add_argument('--sm-hps', type=json.loads, default=os.environ['SM_HPS'])

    # input data and model directories
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.add_argument('--test', type=str, default=os.environ['SM_CHANNEL_TEST'])
    parser.add_argument('--train_file', type=str, default='train_v1.csv')
    parser.add_argument('--test_file', type=str, default='test_v1.csv')

    args, _ = parser.parse_known_args()

    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[label]
    y_test = test_df[label]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",label)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (80%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (20%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
    print("Training RandomForest Model.....")
    print()
    model = RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose=3, n_jobs=None)
    model.fit(X_train, y_train)
    print()

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)


Writing script.py


In [36]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point = "script.py",
    role = "arn:aws:iam::131545665138:role/mobsagemaker",
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="RF-custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600

)

In [37]:
sklearn_estimator.fit({"train":train_path, "test":test_path}, wait = True)

INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2024-05-22-16-37-41-215


2024-05-22 16:37:43 Starting - Starting the training job...
2024-05-22 16:38:00 Starting - Preparing the instances for training...
2024-05-22 16:38:27 Downloading - Downloading input data...
2024-05-22 16:38:58 Downloading - Downloading the training image...
2024-05-22 16:39:48 Training - Training image download completed. Training in progress.
2024-05-22 16:39:48 Uploading - Uploading generated training model2024-05-22 16:39:40,873 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2024-05-22 16:39:40,877 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-22 16:39:40,926 sagemaker_sklearn_container.training INFO     Invoking user training script.
2024-05-22 16:39:41,104 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-22 16:39:41,117 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2024-05-22 16:39:41,131 sagemaker-training-toolkit INFO

In [38]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact present at:", artifact)


2024-05-22 16:40:01 Starting - Preparing the instances for training
2024-05-22 16:40:01 Downloading - Downloading the training image
2024-05-22 16:40:01 Training - Training image download completed. Training in progress.
2024-05-22 16:40:01 Uploading - Uploading generated training model
2024-05-22 16:40:01 Completed - Training job completed
Model artifact present at: s3://sagemaker-us-east-1-131545665138/RF-custom-sklearn-2024-05-22-16-37-41-215/output/model.tar.gz


In [40]:
# Making a copy to built model for deployment
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearn(
    name = model_name,
    model_data=artifact,
    role = "arn:aws:iam::131545665138:role/mobsagemaker",
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION
)

INFO:sagemaker.image_uris:Defaulting to only supported image scope: cpu.


In [41]:
model

In [48]:
# Endpoint Deployment
endpoint_name = "Custom-sklearn-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name
)

EndpointName=Custom-sklearn-model2024-05-22-16-55-07


ValueError: Estimator is not associated with a training job

In [44]:
endpoint_name

'Custom-sklearn-model2024-05-22-16-48-49'

In [45]:

# First five rows of the test dataset for prediction using the endpoint created above
testX[features][0:5].values.tolist()

[[1454.0,
  1.0,
  0.5,
  1.0,
  1.0,
  0.0,
  34.0,
  0.7,
  83.0,
  4.0,
  3.0,
  250.0,
  1033.0,
  3419.0,
  7.0,
  5.0,
  5.0,
  1.0,
  1.0,
  0.0],
 [1092.0,
  1.0,
  0.5,
  1.0,
  10.0,
  0.0,
  11.0,
  0.5,
  167.0,
  3.0,
  14.0,
  468.0,
  571.0,
  737.0,
  14.0,
  4.0,
  11.0,
  0.0,
  1.0,
  0.0],
 [1524.0,
  1.0,
  1.8,
  1.0,
  0.0,
  0.0,
  10.0,
  0.6,
  174.0,
  4.0,
  1.0,
  154.0,
  550.0,
  2678.0,
  16.0,
  5.0,
  13.0,
  1.0,
  0.0,
  1.0],
 [1807.0,
  1.0,
  2.1,
  0.0,
  2.0,
  0.0,
  49.0,
  0.8,
  125.0,
  1.0,
  10.0,
  337.0,
  1384.0,
  1906.0,
  17.0,
  13.0,
  13.0,
  0.0,
  1.0,
  1.0],
 [1086.0,
  1.0,
  1.7,
  1.0,
  0.0,
  1.0,
  43.0,
  0.2,
  111.0,
  6.0,
  1.0,
  56.0,
  1150.0,
  3285.0,
  11.0,
  5.0,
  17.0,
  1.0,
  1.0,
  0.0]]

In [46]:
print(predictor.predict(testX[features][0:5].values.tolist()))

NameError: name 'predictor' is not defined